<font size="5">Imports</font>

In [338]:
import pandas as pd
import numpy as np



<font size="5">Bring in table NHL Stanley Cup Champions from 1927 - 2022.</font>
<br>
<font size="4">Table was scraped from NHL website.</font>
<br>
<font size="4">Source:</font>
<br>
<font size="4">https://www.nhl.com/news/nhl-stanley-cup-champions-winners-complete-list/c-287705398https://www.nhl.com/news/nhl-stanley-cup-champions-winners-complete-list/c-287705398</font>

In [339]:
df_champs = pd.read_csv('NHL_Champions.txt')
df_champs.head(5)

,Year,Team,Coach
0,2022,Colorado Avalanche,Coach Jared Bednar
1,2021,Tampa Bay Lightning,Coach Jon Cooper
2,2020,Tampa Bay Lightning,Coach Jon Cooper
3,2019,St. Louis Blues,Coach Craig Berube
4,2018,Washington Capitals,Coach Barry Trotz


In [340]:
# Have a look at the Championship teams.
print(len(list(set(df_champs['Team']))))
print(sorted(list(set(df_champs['Team']))))

23
['Anaheim Ducks', 'Boston Bruins', 'Calgary Flames', 'Carolina Hurricanes', 'Chicago Black Hawks', 'Chicago Blackhawks', 'Colorado Avalanche', 'Dallas Stars', 'Detroit Red Wings', 'Edmonton Oilers', 'Los Angeles Kings', 'Montreal Canadiens', 'Montreal Maroons', 'New Jersey Devils', 'New York Islanders', 'New York Rangers', 'Ottawa Senators', 'Philadelphia Flyers', 'Pittsburgh Penguins', 'St. Louis Blues', 'Tampa Bay Lightning', 'Toronto Maple Leafs', 'Washington Capitals']


In [341]:
# In df_champs, need to address some name differences.
# Chicago Blackhawks vs Chicago Black Hawks. Chicago should show as having 6 Stanley Cup wins.
# NHL team Montreal Maroons no longer exist, but can leave them in the DataFrame since they did win the Stanley Cup.
df_champs['Team'] = df_champs['Team'].str.replace("Chicago Black Hawks", "Chicago Blackhawks")

<font size="5">Championship teams and the years they've won the Stanley Cup.</font>

In [342]:
df_teams_years_won = df_champs[['Team', 'Year']].copy()
df_teams_years_won = df_teams_years_won.groupby(by=['Team', 'Year']).count() # Empty Dataframe, only MultiIndex remains.

# Argument sort_remaining works left to right. Therefore, need to sort in order of right to left.
df_teams_years_won.sort_index(level=1, ascending=False, inplace=True, sort_remaining=False) # Sort the years, descending.
df_teams_years_won.sort_index(level=0, ascending=True, inplace=True, sort_remaining=False) # Sort the team names, ascending.

# .head(20) to show the new 2022 Champs, the Colorado Avalanche!
df_teams_years_won.head(20)

Empty DataFrame
Columns: []
Index: [(Anaheim Ducks, 2007), (Boston Bruins, 2011), (Boston Bruins, 1972), (Boston Bruins, 1970), (Boston Bruins, 1941), (Boston Bruins, 1939), (Boston Bruins, 1929), (Calgary Flames, 1989), (Carolina Hurricanes, 2006), (Chicago Blackhawks, 2015), (Chicago Blackhawks, 2013), (Chicago Blackhawks, 2010), (Chicago Blackhawks, 1961), (Chicago Blackhawks, 1938), (Chicago Blackhawks, 1934), (Colorado Avalanche, 2022), (Colorado Avalanche, 2001), (Colorado Avalanche, 1996), (Dallas Stars, 1999), (Detroit Red Wings, 2008)]

In [343]:
Championship_Years_filename = 'Championship_Teams_Years.xlsx'
with pd.ExcelWriter(Championship_Years_filename) as writer:
    df_teams_years_won.to_excel(writer, sheet_name='Champions Years')

<font size="5">Championship teams and the years they've won the Stanley Cup.</font>
<br>
<font size="4">Includes subtotal row for count of years won.</font>

In [344]:
df_wins_subtotals = df_champs[['Team', 'Year']].copy()
df_wins_subtotals = df_wins_subtotals.groupby(by=['Team', 'Year']).count() # Empty DataFrame, only multi-index remains.
# Multi-index sort by team asc, year desc.
df_wins_subtotals.sort_index(level=1, ascending=False, inplace=True, sort_remaining=False)
df_wins_subtotals.sort_index(level=0, ascending=True, inplace=True, sort_remaining=False)
# Get a unique list of the Championship teams.
teams_won_list = sorted(list(set([team[0] for team in df_wins_subtotals.index.to_list()])),
                         reverse=False) # Stay consistent to the index sorts.

# Create a Count_Year column from the second level of the multi-index.
df_wins_subtotals['Count_Year'] = [y[1] for y in df_wins_subtotals.index.to_list()]
concat_list = []
for team in teams_won_list:
    # Filter to a Championship team.
    team_rows = df_wins_subtotals.query(f'Team=="{team}"')    
    # print(team_rows)
    # Create an np.nan row.
    subtotal_row = pd.Series([np.nan], index=[f'{team} Total Stanley Cups'], name='Count_Year').to_frame()
    # print(subtotal_row)
    # Fill the np.nan with the selected team's count of years won.
    subtotal_row['Count_Year'].fillna(team_rows['Count_Year'].count(), inplace=True)
    # Concat the filtered team with the subtotal row.
    df_team_subtotal = pd.concat([team_rows, subtotal_row])  
    # Use concat_list to store the filtered team with subtotal row.
    concat_list.append(df_team_subtotal)

# Put it all together.
df_wins_subtotals_2 = pd.concat(concat_list)

# Clean up the results.
df_wins_subtotals_2['Count_Year'] = pd.to_numeric(df_wins_subtotals_2['Count_Year']).astype('int64')

df_wins_subtotals_2['Teams'] = [t[0] if (len(t[0]) > 1) else t for t in df_wins_subtotals_2.index.to_list()]

df_wins_subtotals_2 = df_wins_subtotals_2[['Teams', 'Count_Year']]
df_wins_subtotals_2.reset_index(inplace=True, drop=True)
df_wins_subtotals_2.rename(columns={'Count_Year': 'Years_Won'}, inplace=True)

# .head(25) to see the new Stanley Cup champs and their subtotal of years won!
df_wins_subtotals_2.head(25)


,Teams,Years_Won
0,Anaheim Ducks,2007
1,Anaheim Ducks Total Stanley Cups,1
2,Boston Bruins,2011
3,Boston Bruins,1972
4,Boston Bruins,1970
5,Boston Bruins,1941
6,Boston Bruins,1939
7,Boston Bruins,1929
8,Boston Bruins Total Stanley Cups,6
9,Calgary Flames,1989


In [345]:
# Send the results to excel for end users.
teams_by_years_won_filename = 'teams_by_years_won.xlsx'
with pd.ExcelWriter(teams_by_years_won_filename) as writer:
    df_wins_subtotals_2.to_excel(writer, sheet_name='Teams_Wins', index=False)

<font size="5">Which Team has the most Stanley Cup wins?</font>

In [346]:
df_champs_wins = df_champs[['Team', 'Year']].copy()
df_champs_wins.rename(columns={'Year':'Stanley_Cup_Wins'}, inplace=True)
df_champs_wins = df_champs_wins.groupby(by=['Team'], as_index=False).count()
df_champs_wins.sort_values(by=['Stanley_Cup_Wins'], ascending=False, inplace=True)
df_champs_wins.reset_index(drop=True, inplace=True)
df_champs_wins.head(10)

,Team,Stanley_Cup_Wins
0,Montreal Canadiens,22
1,Toronto Maple Leafs,11
2,Detroit Red Wings,11
3,Chicago Blackhawks,6
4,Boston Bruins,6
5,Edmonton Oilers,5
6,Pittsburgh Penguins,5
7,New York Islanders,4
8,New York Rangers,4
9,Tampa Bay Lightning,3


<font size="5">Of the current 2022 NHL teams, which team(s) have yet to win a Stanley Cup?</font>
<br>
<font size="4">Current teams list scraped from NHL website.</font>
<br>
<font size="4">Source:</font>
<br>
<font size="4">https://www.nhl.com/info/teamshttps://www.nhl.com/info/teams</font>

In [347]:
df_current_teams = pd.read_csv('NHL_Teams.csv',
                              usecols=['NHL_Teams'])
# Stay consistent with previous sorting.
df_current_teams.sort_values(by=['NHL_Teams'], inplace=True)
df_current_teams.reset_index(inplace=True, drop=True)
print(df_current_teams.shape) # 32 teams, as expected.
df_current_teams.head(5)

(32, 1)


,NHL_Teams
0,Anaheim Ducks
1,Arizona Coyotes
2,Boston Bruins
3,Buffalo Sabres
4,Calgary Flames


In [348]:
# In df_current_teams, change Montréal Canadiens to Montreal Canadiens, for simplicity. (I don't have é on my keyboard.)
df_current_teams['NHL_Teams'] = df_current_teams['NHL_Teams'].str.replace("Montréal Canadiens", "Montreal Canadiens")

In [349]:
df_champs.head(5)

,Year,Team,Coach
0,2022,Colorado Avalanche,Coach Jared Bednar
1,2021,Tampa Bay Lightning,Coach Jon Cooper
2,2020,Tampa Bay Lightning,Coach Jon Cooper
3,2019,St. Louis Blues,Coach Craig Berube
4,2018,Washington Capitals,Coach Barry Trotz


In [350]:
current_teams = list(df_current_teams['NHL_Teams'])
champ_teams = list(df_champs['Team'])

teams_never_won = []
num = 1
print("Current NHL teams that have yet to win a Stanley Cup:")
for team in current_teams:
    if team not in champ_teams:
        print(f'{num}. {team}')
        teams_never_won.append(team)
        num+=1
        
df_never_won = pd.DataFrame(data={'Teams_Never_Won': teams_never_won})

Current NHL teams that have yet to win a Stanley Cup:
1. Arizona Coyotes
2. Buffalo Sabres
3. Columbus Blue Jackets
4. Florida Panthers
5. Minnesota Wild
6. Nashville Predators
7. San Jose Sharks
8. Seattle Kraken
9. Vancouver Canucks
10. Vegas Golden Knights
11. Winnipeg Jets


<font size="3">Check the results.</font>

In [351]:
count_won = len(list(set(champ_teams)))
count_never_won = len(list(df_never_won['Teams_Never_Won']))
count_current_teams = len(list(set(current_teams)))
# We would expect that won + never won = total teams
print(count_won)
print(count_never_won)
print(count_never_won + count_won)

22
11
33


<font size="3">22 won + 11 never won = 33 teams; however there are only 32 current teams.</font>
<br> 
<font size="4">&emsp;&rarr;</font>
<font size="3">There is one Championship team from 1927 - 2022 that is not a current team.</font>

<font size="4">Of all 22 teams to win the Stanley Cup since 1927, the only team no longer in existence is the Montreal Maroons.</font>

In [352]:
# Only Montreal Maroons are in the Stanley Cup winners list, but not in the current teams list.
for team in list(set(champ_teams)):
    if team not in current_teams:
        print(team)

Montreal Maroons


<font size="5">Of all current NHL teams that have won the Stanley Cup, in what year was their most recent win?</font>

In [353]:
df_champs_distinct = df_champs.copy()
df_champs_distinct.drop_duplicates(subset=['Team'], inplace=True)
print(df_champs_distinct.shape)
# Merging here will remove Montreal Maroons, as they are not a current NHL team.
df_champs_merge = pd.merge(df_current_teams, df_champs_distinct, how='inner', left_on=['NHL_Teams'], right_on=['Team'])
print(df_champs_merge.shape)
# Sort by Year this time, to see the most recent year that a team has won the Stanley Cup.
df_champs_merge.sort_values(by=['Year'], ascending=False, inplace=True)
df_champs_merge.drop(columns=['Team'], inplace=True)
df_champs_merge = df_champs_merge[['NHL_Teams','Year','Coach']]
df_champs_merge.rename(columns={'NHL_Teams': 'Current_NHL_Teams',                                
                                'Year': 'Year_Most_Recently_Won',
                                'Coach': 'Coached_By'},                   
                       inplace=True)                             
                       
df_champs_merge.reset_index(inplace=True, drop=True)
df_champs_merge.head(30)

(22, 3)
(21, 4)


,Current_NHL_Teams,Year_Most_Recently_Won,Coached_By
0,Colorado Avalanche,2022,Coach Jared Bednar
1,Tampa Bay Lightning,2021,Coach Jon Cooper
2,St. Louis Blues,2019,Coach Craig Berube
3,Washington Capitals,2018,Coach Barry Trotz
4,Pittsburgh Penguins,2017,Coach Mike Sullivan
5,Chicago Blackhawks,2015,Coach Joel Quenneville
6,Los Angeles Kings,2014,Coach Darryl Sutter
7,Boston Bruins,2011,Coach Claude Julien
8,Detroit Red Wings,2008,Coach Mike Babcock
9,Anaheim Ducks,2007,Coach Randy Carlyle


<font size="5">Summarize all current NHL teams by year of most recent Stanley Cup victory.</font>

In [354]:
df_never_won.head(2)

,Teams_Never_Won
0,Arizona Coyotes
1,Buffalo Sabres


In [355]:
# Rename this column for pd.concat() purposes.
df_never_won.rename(columns={'Teams_Never_Won': 'Current_NHL_Teams'}, inplace=True)
df_never_won.head(2)

,Current_NHL_Teams
0,Arizona Coyotes
1,Buffalo Sabres


In [356]:
df_won_and_not_won = pd.concat([df_champs_merge, df_never_won])

df_won_and_not_won.fillna(-1, inplace=True)
df_won_and_not_won['Year_Most_Recently_Won'] = pd.to_numeric(df_won_and_not_won['Year_Most_Recently_Won']).astype('int64', errors='ignore')

df_won_and_not_won.loc[df_won_and_not_won['Year_Most_Recently_Won'] < 0, ['Year_Most_Recently_Won', 'Coached_By']]  = "Yet to Win"

print(df_won_and_not_won.shape) # 32 rows, as expected.
df_won_and_not_won.head(32)

(32, 3)


,Current_NHL_Teams,Year_Most_Recently_Won,Coached_By
0,Colorado Avalanche,2022,Coach Jared Bednar
1,Tampa Bay Lightning,2021,Coach Jon Cooper
2,St. Louis Blues,2019,Coach Craig Berube
3,Washington Capitals,2018,Coach Barry Trotz
4,Pittsburgh Penguins,2017,Coach Mike Sullivan
5,Chicago Blackhawks,2015,Coach Joel Quenneville
6,Los Angeles Kings,2014,Coach Darryl Sutter
7,Boston Bruins,2011,Coach Claude Julien
8,Detroit Red Wings,2008,Coach Mike Babcock
9,Anaheim Ducks,2007,Coach Randy Carlyle


In [357]:
# Send to excel for end users.
teams_wins_summary_filename = 'Current_Teams_Stanley_Cup_Championships_Summary.xlsx'
with pd.ExcelWriter(teams_wins_summary_filename) as writer:
    df_won_and_not_won.to_excel(writer, sheet_name='Cup_Summary')

<font size="5">Fun Fact: 2004-2005 NHL season was cancelled, since players and league could not agree on salary terms.</font>

In [358]:
print(list(df_champs['Year']))

[2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979, 1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1969, 1968, 1967, 1966, 1965, 1964, 1963, 1962, 1961, 1960, 1959, 1958, 1957, 1956, 1955, 1954, 1953, 1952, 1951, 1950, 1949, 1948, 1947, 1946, 1945, 1944, 1943, 1942, 1941, 1940, 1939, 1938, 1937, 1936, 1935, 1934, 1933, 1932, 1931, 1930, 1929, 1928, 1927]
